## MobileNet V1 model for training Food101 datasets
* number of parameters: 3,380,965

In [15]:
import torch

from torchvision import transforms, datasets

from pathlib import Path

import sys
sys.path.append('..')

from python_scripts import model_builder, engine, data_setup

In [16]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [17]:
food101_transforms_train = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.Normalize(
      [0.485, 0.456, 0.406],
      [0.229, 0.224, 0.225]
    )
])

food101_transforms_test = transforms.Compose([
    transforms.Resize(225),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
      [0.485, 0.456, 0.406],
      [0.229, 0.224, 0.225]
    )
])

In [18]:
learning_rate_list = [1e-4]
weight_decay_list = [1e-4]
epochs_list = [10]
batch_size_list = [16]

In [19]:
data_dir = Path('../data/extracted')

train_data_food101 = datasets.Food101(
    root=data_dir,
    split='train',
    transform=food101_transforms_train,
    download=True
)
test_data_food101 = datasets.Food101(
    root=data_dir,
    split='test',
    transform=food101_transforms_test,
    download=True
)

class_names_food101 = train_data_food101.classes

In [20]:
def model_generator(weights=None):
    return model_builder.MobileNetV1(num_classes=len(class_names_food101))

In [21]:
train_dataset, _ = data_setup.split_dataset(
    dataset=train_data_food101,
    split_size=1, # you can start with small number of train set to check normality at first
    seed=42
)
test_dataset, _ = data_setup.split_dataset(
    dataset=test_data_food101,
    split_size=1,
    seed=42
)

tuning_results = engine.HP_tune_train(
    model=None,
    model_generator=model_generator,
    model_weights=None,
    model_name='MobileNetV1_food101',
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_tensorboard_writer=False,
    device=device,
    gradient_accumulation_num=1
)

[INFO] Splitting dataset of length 75750 into splits of size: 75750 and 0
[INFO] Splitting dataset of length 25250 into splits of size: 25250 and 0


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0 | Train_loss: 4.5410, Train_acc: 0.0235 | Test_loss: 4.2319, Test_acc: 0.0595
Epoch: 1 | Train_loss: 4.3856, Train_acc: 0.0447 | Test_loss: 4.0223, Test_acc: 0.0852
Epoch: 2 | Train_loss: 4.2468, Train_acc: 0.0639 | Test_loss: 3.7975, Test_acc: 0.1231
Epoch: 3 | Train_loss: 4.1196, Train_acc: 0.0839 | Test_loss: 3.5985, Test_acc: 0.1624
Epoch: 4 | Train_loss: 4.0126, Train_acc: 0.1029 | Test_loss: 3.3924, Test_acc: 0.2029
Epoch: 5 | Train_loss: 3.9051, Train_acc: 0.1210 | Test_loss: 3.2380, Test_acc: 0.2285
Epoch: 6 | Train_loss: 3.8035, Train_acc: 0.1393 | Test_loss: 3.1324, Test_acc: 0.2421
Epoch: 7 | Train_loss: 3.6993, Train_acc: 0.1575 | Test_loss: 2.9437, Test_acc: 0.2803
Epoch: 8 | Train_loss: 3.5898, Train_acc: 0.1779 | Test_loss: 2.7403, Test_acc: 0.3243
Epoch: 9 | Train_loss: 3.4946, Train_acc: 0.1965 | Test_loss: 2.6434, Test_acc: 0.3442
